 * trailer: 
     * show a histogram with examples of distance / orientation with samples over ALL datasets
     * show perf!  show time to fetch frames using Spark + Parquet
     * show a video of one camera with debug overlays.  maybe one with delauny lidar too (!)
     * show a frame HTML with 3d interface
     * show new things: argo associated bikes, delauny lidar, occlusion tree
 * supported datasets, how to get a blurb and **stats** on each of them.  prolly render histo reports for each.
 * data structures:
    * StampedDatum
    * Frame

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/psegs')

In [4]:
from psegs.datasets import kitti

kitti.DSUtil.test()

[autoreload of psegs.datasets.kitti failed: Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/lib/python3/dist-packages/IPython/extensions/autoreload.py", line 292, in update_class
    if (old_obj == new_obj) is True:
  File "<attrs generated eq attr._make.Attribute>", line 4, in __eq__
    return  (
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
]
2020-09-28 00:29:47,099	oarph 2157 : Running cd /opt/psegs && pytest -s -vvv -k test_kitti ...


KeyboardInterrupt: 

In [1]:
import sys
sys.path.append('/opt/psegs')
from oarphpy.spark import NBSpark
NBSpark.SRC_ROOT = '/opt/psegs'
NBSpark.SRC_ROOT_MODULES = ['psegs']
NBSpark.MAYBE_REBUILD_EGG_EVERY_CELL_RUN = False
NBSpark.CONF_KV = {
    'spark.driver.memory': '8g',
    'spark.pyspark.python': 'python3',
    'spark.python.worker.reuse': False,
    'spark.sql.files.maxPartitionBytes': int(8 * 1e6),
    'spark.port.maxRetries': '256',
  }
from psegs.datasets import kitti

spark = NBSpark.getOrCreate()
df = kitti.KITTISDTable.as_df(spark)
# df = spark.read.parquet('/tmp/avs_test/test_kitti_sd_table_tracking/sd_table/')
df.show(5)
# print(df.count())

2020-09-27 22:49:30,378	oarph 40 : Using source root /opt/psegs 
2020-09-27 22:49:30,379	oarph 40 : Using source root /opt 
2020-09-27 22:49:30,547	oarph 40 : Generating egg to /tmp/tmpb7yp4q0b_oarphpy_eggbuild ...
2020-09-27 22:49:30,561	oarph 40 : ... done.  Egg at /tmp/tmpb7yp4q0b_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg
2020-09-27 22:49:34,166	ps   40 : Loading /opt/psegs/dataroot/stamped_datum/stamped_datums ...


AnalysisException: Path does not exist: file:/opt/psegs/dataroot/stamped_datum/stamped_datums;

In [9]:
# seg = df.filter('uri.segment_id = "kitti-tracking-train-0009"')

In [10]:
# seg_uri = seg.select('uri').persist()
df.createOrReplaceTempView('data')
# seg_uri.show()

In [11]:
spark.sql("""
    SELECT
      uri.segment_id AS seg, 
      uri.topic AS topic,
      count(*) AS N,
      MAX(uri.timestamp),
      MIN(uri.timestamp),
      (MAX(uri.timestamp) - MIN(uri.timestamp)) * 1e-9 AS len
    FROM data
    GROUP BY topic, seg
    ORDER BY len ASC
""").show(1000, truncate=False)

+-------------------------+--------+----+---------------------------------+---------------------------------+-------------+
|seg                      |topic   |N   |max(uri.timestamp AS `timestamp`)|min(uri.timestamp AS `timestamp`)|len          |
+-------------------------+--------+----+---------------------------------+---------------------------------+-------------+
|kitti-tracking-train-0012|ego_pose|78  |1317047690486674021              |1317047682506437291              |7.980236730  |
|kitti-tracking-test-0027 |ego_pose|85  |9400000000                       |1000000000                       |8.400000000  |
|kitti-tracking-train-0014|ego_pose|106 |1317212788870919845              |1317212777989915167              |10.881004678 |
|kitti-tracking-test-0006 |ego_pose|114 |12300000000                      |1000000000                       |11.300000000 |
|kitti-tracking-test-0001 |ego_pose|147 |15600000000                      |1000000000                       |14.600000000 |
|kitti-t